In [8]:
%reset -f

import os
import numpy as np
import numba as  nb
import pandas as pd


import plotly.graph_objects as go
from plotly.graph_objects import Figure

from plotly.graph_objects import Scatter
from plotly.graph_objects import Ohlc

from sxo.util.binning import bin_values
from sxo.util.runtime.timeseries import RedisTs

In [9]:
os.environ["REDIS_PASS"] = "boss"

In [10]:
ts_bid = RedisTs('ts_bid_FxSpot_GBPEUR')
ts_ask = RedisTs('ts_ask_FxSpot_GBPEUR')
ts_bsz = RedisTs('ts_bsz_FxSpot_GBPEUR')
ts_asz = RedisTs('ts_asz_FxSpot_GBPEUR')
t0 = np.datetime64('2023-10-08 00:00:00').astype('datetime64[ms]').astype(int)
bid = ts_bid.get_range(name='bid',)
ask = ts_ask.get_range(name='ask')
bsz = ts_bid.get_range(name='bsz')
asz = ts_ask.get_range(name='asz')
bid_ask = bid.join(ask, how='inner').join(bsz, how='inner').join(asz, how='inner').reset_index()
bid_ask['t'] = bid_ask['t'].values.astype('datetime64[ms]').astype('datetime64[ns]')


In [11]:
bid_ask['mid'] = (bid_ask['bid'].values + bid_ask['ask'].values) / 2

In [12]:
bid_ask.to_parquet('tmp.parq')
#bid_ask = pd.read_parquet('tmp.parq')

In [13]:

bins = bin_values(bid_ask, 60, time_col='t', value_col='mid')
bins.tail(4)

,bin_start,bin_end,t0,t1,n_samples,o,h,l,c,twap
3243,2023-10-11 19:20:00,2023-10-11 19:21:00,2023-10-11 19:20:00.044,2023-10-11 19:20:58.101,52,1.160025,1.160035,1.159995,1.159995,1.160023
3244,2023-10-11 19:21:00,2023-10-11 19:22:00,2023-10-11 19:21:00.606,2023-10-11 19:21:50.160,41,1.159995,1.159995,1.159935,1.159935,1.159946
3245,2023-10-12 08:07:00,2023-10-12 08:08:00,2023-10-12 08:07:47.246,2023-10-12 08:07:58.930,12,1.157735,1.157805,1.157735,1.157805,1.157768
3246,2023-10-12 08:08:00,2023-10-12 08:09:00,2023-10-12 08:08:00.128,2023-10-12 08:08:03.628,4,1.157805,1.157825,1.157805,1.157805,1.157805


In [14]:
import datetime as dt

df_plt = bid_ask.copy()[bid_ask.t > np.datetime64('2023-07-06 06:00:00')]
bin_plt = bins.copy()[bins.bin_start > np.datetime64('2023-07-06 12:00:00')]
m = 15
# fig = Figure(layout=go.Layout(height=500, margin={'t':m,'b':m,'l':m,'r':m}))
# fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['ask'], name='ask', mode="markers", marker={'size':2},showlegend=False))
# fig.add_traces(Scatter(x=df_plt['t'], y=df_plt['bid'], name='bid', mode="markers", marker={'size':2},showlegend=False))
#fig.add_traces(Scatter(x=bin_plt['bin_start'], y=bin_plt['twap'], name='bid', showlegend=False))

# s, e, h, l  = bin_plt['bin_start'].values.astype(pd.Timestamp), bin_plt['bin_end'].values.astype(pd.Timestamp), bin_plt['h'].values, bin_plt['l'].values
# i = 3
# fig.add_traces(Scatter(x=[s[i], s[i], e[i], e[i], s[i]], y=[l[i], h[i], h[i], l[i], l[i]], fill='toself'))


fig = go.Figure(data=go.Ohlc(x=bin_plt['bin_start'],
                    open=bin_plt['o'],
                    high=bin_plt['h'],
                    low=bin_plt['l'],
                    close=bin_plt['c']))

# go.Ohlc(x=bin_plt['Date'],
#                     open=df['AAPL.Open'],
#                     high=df['AAPL.High'],
#                     low=df['AAPL.Low'],
#                     close=df['AAPL.Close']))

#fig.add_traces(Scatter(x=df_plt['t'], y=(df_plt['bid'] + df_plt['ask'])/2, name='mid', mode="markers", marker={'size':2, 'color':'black'},showlegend=False))
fig.show()

/home/iztok/.conda/envs/sxo-research3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

